<a href="https://colab.research.google.com/github/potat0w/Aaagh-more-math/blob/main/Final_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


df = pd.read_csv("Social_Network_Ads.csv")
df.head()


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [ ]:
df.shape


(400, 5)

## 2.Data preprocessing

In [ ]:

le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])


In [ ]:
df.drop(columns=['User ID'], inplace=True)

In [ ]:
def remove_outliers(data, columns):

    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR

        outliers = data[(data[col] < lower_limit) | (data[col] > upper_limit)]
        num_outliers = outliers.shape[0]

        print(f"{col}: {num_outliers} outliers found")
        data = data[(data[col] >= lower_limit) & (data[col] <= upper_limit)]

    return data


In [ ]:
df = remove_outliers(df, ['Age', 'EstimatedSalary'])

Age: 0 outliers found
EstimatedSalary: 0 outliers found


In [ ]:
X = df.drop('Purchased', axis=1)
y = df['Purchased']


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)


## 3.pipeline *creation*

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])


## 4.Primary Model Selection

Logistic Regression

Justification:

This is a binary classification problem as target variable is binary.
Dataset is small and linearly separable.


## 5.Model training

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LogisticRegression())])

## 6.Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

cv_scores = cross_val_score(
    pipeline, X_train, y_train, cv=5, scoring='accuracy'
)

print("Cross-validation scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())
print("Standard Deviation:", cv_scores.std())


Cross-validation scores: [0.83333333 0.85       0.8        0.78333333 0.81666667]
Mean Accuracy: 0.8166666666666667
Standard Deviation: 0.02357022603955158


## 7.Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__C': [0.01, 0.1, 1, 10, 100],
    'model__solver': ['liblinear', 'lbfgs']
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='accuracy'
)

grid.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', LogisticRegression())]),
             param_grid={'model__C': [0.01, 0.1, 1, 10, 100],
                         'model__solver': ['liblinear', 'lbfgs']},
             scoring='accuracy')

In [ ]:
# Best parameters
grid.best_params_


{'model__C': 0.01, 'model__solver': 'liblinear'}

In [ ]:
# Best cross-validation score
grid.best_score_


np.float64(0.8233333333333333)

## 8.Best model selection

In [ ]:
best_model = grid.best_estimator_


## 9.evaluation

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = best_model.predict(X_test)
accuracy_score(y_test, y_pred)


0.86

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[61,  2],
       [12, 25]])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.84      0.97      0.90        63
           1       0.93      0.68      0.78        37

    accuracy                           0.86       100
   macro avg       0.88      0.82      0.84       100
weighted avg       0.87      0.86      0.85       100



In [ ]:
import joblib

joblib.dump(best_model, "model.pkl")


['model.pkl']